# Graph Clustering

In [1]:
import os
import igraph as ig
import pandas as pd
import sys
sys.path.append(os.path.abspath(".."))

import plotly.plotly as py
from sklearn import metrics
from plotly.graph_objs import *
from url2vec.util.plotter import *
from url2vec.util.metrics import *
from __future__ import print_function
from url2vec.util.seqmanager import *

In [2]:
# available datasets
# cs.illinois.edu    cs.stanford.edu    eecs.mit.edu    cs.princeton.edu    cs.ox.ac.uk
site = "cs.illinois.edu"

The crawling proccess has been done in two different ways:
- **No costraint**: the crawler follows a random outlink from all of the outlinks in a given page
- **List costraint**: the crawler follows a random outlink but only from the outlinks in "lists"

---

## Graph - no-costraint 

Here we're loading all the files that the crawler has generated for creating the web graph.

See the [Dataset README](https://github.com/chrisPiemonte/url2vec/tree/master/dataset "Dataset") for further information.

In [3]:
nocostraint_path = os.getcwd() + "/../dataset/" + site + "/no_constraint/words1000_depth10/"
nocostraint_urlmap_path = nocostraint_path + "urlsMap.txt"
nocostraint_edges_path  = nocostraint_path + "edges.txt"

codeurlmap_nocostraint  = get_urlmap(nocostraint_urlmap_path)
nocostraint_graph       = graph_from_file(nocostraint_edges_path, codeurlmap_nocostraint)

print("Number of vertices:", len(nocostraint_graph.vs))
print("Number of edges:",    len(nocostraint_graph.es))
print("\n\n")

Number of vertices: 4183
Number of edges: 27954





The test dataset has been manually clustered for evaluation purposes, here some attributes are assigned to the vertices, such as: 
- **color: based** on the cluster membership of the URL
- **true_label**: the cluster membership

In [5]:
gt = GroundTruth(os.getcwd() + "/../dataset/" + site + "/ground_truth/urlToMembership.txt")
# deletes vertices that don't appear in at least one sequence
nocostraint_graph.delete_vertices([vertex.index for vertex in nocostraint_graph.vs if vertex["name"] == "missing"])

for vertex in nocostraint_graph.vs:
    vertex["color"] = get_color(int(gt.get_groundtruth(vertex["name"])))
    vertex["true_label"] = int(gt.get_groundtruth(vertex["name"]))

Url not found
Url not found
Url not found
Url not found


Plotting the no-costraint graph manually clusterized:

In [6]:
nocostraint_fig = graph3d_plot(nocostraint_graph, "No-costraint Network - Manually clusterized")
py.iplot(nocostraint_fig, filename="No-costraint Network - Manually clusterized")

<div>
    <a href="https://plot.ly/~chrispolo/44/" target="_blank" title="No-costraint network" style="display: block; text-align: center;"><img src="https://plot.ly/~chrispolo/44.png" alt="No-costraint network" style="max-width: 100%;width: 1000px;"  width="1000" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="chrispolo:44"  src="https://plot.ly/embed.js" async></script>
</div>


### Clustering - No-Costraint Graph

Two clustering algorithm are reported:
- **fastgreedy**: is a hierarchical approach, but it is bottom-up instead of top-down. It tries to optimize a quality function called modularity in a greedy manner. Initially, every vertex belongs to a separate community, and communities are merged iteratively such that each merge is locally optimal (i.e. yields the largest increase in the current value of modularity). The algorithm stops when it is not possible to increase the modularity any more, so it gives you a grouping as well as a dendrogram. The method is fast and it is the method that is usually tried as a first approximation because it has no parameters to tune. However, it is known to suffer from a resolution limit, i.e. communities below a given size threshold (depending on the number of nodes and edges if I remember correctly) will always be merged with neighboring communities.


- **walktrap**: is an approach based on random walks. The general idea is that if you perform random walks on the graph, then the walks are more likely to stay within the same community because there are only a few edges that lead outside a given community. Walktrap runs short random walks of 3-4-5 steps (depending on one of its parameters) and uses the results of these random walks to merge separate communities in a bottom-up manner like fastgreedy.community. Again, you can use the modularity score to select where to cut the dendrogram. It is a bit slower than the fast greedy approach but also a bit more accurate

In [7]:
# fastgreedy 
%time fastgreedy_dendogram = nocostraint_graph.community_fastgreedy()
fastgreedy_clustering = fastgreedy_dendogram.as_clustering(16)

print("Fastgreedy clusters sizes:", fastgreedy_clustering.sizes())

# walktrap 
%time walktrap_dendogram = nocostraint_graph.community_walktrap(steps=3)
walktrap_clustering = walktrap_dendogram.as_clustering(16)

print("Walktrap clusters sizes:", walktrap_clustering.sizes())
print("\n\n")

Fastgreedy clusters sizes: [838, 1231, 597, 413, 24, 171, 213, 35, 79, 32, 21, 104, 41, 28, 26, 98]
Walktrap clusters sizes: [3445, 23, 13, 8, 5, 5, 32, 13, 16, 19, 7, 73, 8, 246, 12, 26]





Assigning some attributes to graph vertices for evaluation and visualization purposes

In [8]:
for i in range(len(walktrap_clustering.membership)):
    nocostraint_graph.vs[i]["color"] = get_color(walktrap_clustering.membership[i])
    nocostraint_graph.vs[i]["pred_walktrap_label"] = walktrap_clustering.membership[i]
    nocostraint_graph.vs[i]["pred_fastgreedy_label"] = fastgreedy_clustering.membership[i]

Plotting the no-costraint graph, colored with the walktrap membership:

In [9]:
nocostraint_fig = graph3d_plot(nocostraint_graph, "No-costraint Network - WalkTrap clustered")
py.iplot(nocostraint_fig, filename="No-costraint Network - WalkTrap clustered")

<div>
    <a href="https://plot.ly/~chrispolo/46/" target="_blank" title="No-costraint Network - WalkTrap clusterized" style="display: block; text-align: center;"><img src="https://plot.ly/~chrispolo/46.png" alt="No-costraint Network - WalkTrap clusterized" style="max-width: 100%;width: 1000px;"  width="1000" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="chrispolo:46"  src="https://plot.ly/embed.js" async></script>
</div>


---

## Graph - list-costraint

Here we're loading all the files that the crawler has generated for creating the web graph.
See the [Dataset](https://github.com/chrisPiemonte/url2vec/tree/master/dataset "Dataset") README for further information.

In [10]:
listcostraint_path        = os.getcwd() + "/../dataset/" + site + "/list_constraint/words1000_depth10/"
listcostraint_urlmap_path = listcostraint_path + "urlsMap.txt"
listcostraint_edges_path  = listcostraint_path + "edges.txt"

urlmap_listcostraint      = get_urlmap(listcostraint_urlmap_path)
listcostraint_graph       = graph_from_file(listcostraint_edges_path, urlmap_listcostraint)

print("Number of vertices:", len(listcostraint_graph.vs))
print("Number of edges:",    len(listcostraint_graph.es))
print("\n\n")

Number of vertices: 3821
Number of edges: 24374





The test dataset has been manually clustered for evaluation purposes, here some attributes are assigned to the vertices, such as: 
- **color: based** on the cluster membership of the URL
- **true_label**: the cluster membership

In [11]:
gt = GroundTruth(os.getcwd() + "/../dataset/" + site + "/ground_truth/urlToMembership.txt")
listcostraint_graph.delete_vertices([vertex.index for vertex in listcostraint_graph.vs if vertex["name"] == "missing"])

for vertex in listcostraint_graph.vs:
    vertex["color"] = get_color(int(gt.get_groundtruth(vertex["name"])))
    vertex["true_label"] = int(gt.get_groundtruth(vertex["name"]))

Url not found
Url not found
Url not found
Url not found


Plotting the list-costraint graph manually clustered:

In [12]:
listcostraint_fig = graph3d_plot(listcostraint_graph, "List-costraint Network - Manually clustered")
py.iplot(listcostraint_fig, filename="List-costraint Network - Manually clustered")

<div>
    <a href="https://plot.ly/~chrispolo/50/" target="_blank" title="List-costraint Network - Manually clusterized" style="display: block; text-align: center;"><img src="https://plot.ly/~chrispolo/50.png" alt="List-costraint Network - Manually clusterized" style="max-width: 100%;width: 1000px;"  width="1000" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="chrispolo:50"  src="https://plot.ly/embed.js" async></script>
</div>


### Clustering - List-Costraint Graph

Two clustering algorithm are reported:
- **fastgreedy**: is a hierarchical approach, but it is bottom-up instead of top-down. It tries to optimize a quality function called modularity in a greedy manner. Initially, every vertex belongs to a separate community, and communities are merged iteratively such that each merge is locally optimal (i.e. yields the largest increase in the current value of modularity). The algorithm stops when it is not possible to increase the modularity any more, so it gives you a grouping as well as a dendrogram. The method is fast and it is the method that is usually tried as a first approximation because it has no parameters to tune. However, it is known to suffer from a resolution limit, i.e. communities below a given size threshold (depending on the number of nodes and edges if I remember correctly) will always be merged with neighboring communities.


- **walktrap**: is an approach based on random walks. The general idea is that if you perform random walks on the graph, then the walks are more likely to stay within the same community because there are only a few edges that lead outside a given community. Walktrap runs short random walks of 3-4-5 steps (depending on one of its parameters) and uses the results of these random walks to merge separate communities in a bottom-up manner like fastgreedy.community. Again, you can use the modularity score to select where to cut the dendrogram. It is a bit slower than the fast greedy approach but also a bit more accurate

In [13]:
%time fastgreedy_lc_dendogram = listcostraint_graph.community_fastgreedy()
fastgreedy_lc_clustering = fastgreedy_lc_dendogram.as_clustering(16)

print("Fastgreedy clusters sizes:", fastgreedy_lc_clustering.sizes())

%time walktrap_lc_dendogram = listcostraint_graph.community_walktrap(steps=3)
walktrap_lc_clustering = walktrap_lc_dendogram.as_clustering(16)

print("Walktrap clusters sizes:", walktrap_lc_clustering.sizes())
print("\n\n")

Fastgreedy clusters sizes: [896, 862, 137, 324, 374, 65, 32, 12, 6, 89, 103, 221, 25, 25, 38, 70]
Walktrap clusters sizes: [2767, 72, 32, 12, 6, 4, 68, 97, 51, 119, 3, 3, 7, 21, 2, 15]





Assigning some attributes to graph vertices for evaluation and visualization purposes

In [14]:
for i in range(len(walktrap_lc_clustering.membership)):
    listcostraint_graph.vs[i]["color"] = get_color(walktrap_lc_clustering.membership[i])
    listcostraint_graph.vs[i]["pred_walktrap_label"] = walktrap_lc_clustering.membership[i]
    listcostraint_graph.vs[i]["pred_fastgreedy_label"] = fastgreedy_lc_clustering.membership[i]
    
for vertex in listcostraint_graph.vs:
    if vertex["pred_fastgreedy_label"] == None:
        vertex["pred_fastgreedy_label"] = -1

Plotting the list-costraint graph, colored with the walktrap membership:

In [15]:
listcostraint_fig = graph3d_plot(listcostraint_graph, "List-costraint Network - WalkTrap clustered")
py.iplot(listcostraint_fig, filename="List-costraint Network - WalkTrap clustered")

<div>
    <a href="https://plot.ly/~chrispolo/52/" target="_blank" title="List-costraint Network - WalkTrap clusterized" style="display: block; text-align: center;"><img src="https://plot.ly/~chrispolo/52.png" alt="List-costraint Network - WalkTrap clusterized" style="max-width: 100%;width: 1000px;"  width="1000" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="chrispolo:52"  src="https://plot.ly/embed.js" async></script>
</div>

### Evaluation
See the [scikit-learn documentaion](http://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation "ti") for futher information

- **Confusion matrix**: Each column of the matrix represents the instances in a predicted cluster while each row represents the instances in an actual cluster

#### Confusion table - Waltrap on No-costraint graph

In [16]:
nocostraint_conftable_df = pd.DataFrame(
    get_confusion_table(nocostraint_graph.vs["true_label"], nocostraint_graph.vs["pred_walktrap_label"]),
    index=set(nocostraint_graph.vs["true_label"]),
    columns=set(nocostraint_graph.vs["pred_walktrap_label"])
)

nocostraint_conftable_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1460,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,13,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,19,0,0,0,0,0,0
6,1,0,0,0,5,0,0,0,0,0,6,0,0,0,0,0
7,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,73,0,0,0,0
9,0,0,0,0,0,0,31,0,0,0,0,0,0,0,0,0


#### Confusion table - Waltrap on List-costraint graph

In [17]:
listcostraint_conftable_df = pd.DataFrame(
    get_confusion_table(listcostraint_graph.vs["true_label"], listcostraint_graph.vs["pred_fastgreedy_label"]), 
    index=set(listcostraint_graph.vs["true_label"]),
    columns=set(listcostraint_graph.vs["pred_fastgreedy_label"]))

listcostraint_conftable_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,497,358,0,6,234,28,0,0,0,1,24,220,16,0,0,0
1,0,1,16,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,45,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,12,0,0,0,0,0,0,0,0
6,1,1,0,0,0,0,0,0,6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,84,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,68,0,0,0,0,0
9,0,0,0,0,0,0,31,0,0,0,0,0,0,0,0,0
10,40,37,0,0,1,0,0,0,0,0,0,0,0,0,0,0
11,70,400,3,5,66,12,0,0,0,1,4,0,3,0,21,0


### Other metrics:

- **Homogeneity**: each cluster contains only members of a single class



- **completeness**: all members of a given class are assigned to the same cluster



- **V-measure**: The V-measure is actually equivalent to the mutual information (NMI) discussed above normalized by the sum of the label entropies


- **Adjusted Rand index**: Given the knowledge of the *ground truth* class assignments *true_label* and our clustering algorithm assignments of the same samples *pred_walktrap_label*, the adjusted Rand index is a function that measures the similarity of the two assignments, ignoring permutations and with chance normalization


- **Mutual Information based scores**: Given the knowledge of the ground truth class assignments labels_true and our clustering algorithm assignments of the same samples labels_pred, the Mutual Information is a function that measures the agreement of the two assignments, ignoring permutations. Two different normalized versions of this measure are available, Normalized Mutual Information(NMI) and Adjusted Mutual Information(AMI). NMI is often used in the literature while AMI was proposed more recently and is normalized against chance

In [18]:
true_label_lc = listcostraint_graph.vs["true_label"]
true_label_nc = nocostraint_graph.vs["true_label"]

pred_walktrap_label_nc = nocostraint_graph.vs["pred_walktrap_label"]
pred_fastgreedy_label_nc = nocostraint_graph.vs["pred_fastgreedy_label"]

pred_walktrap_label_lc = listcostraint_graph.vs["pred_walktrap_label"]
pred_fastgreedy_label_lc = listcostraint_graph.vs["pred_fastgreedy_label"]

metrics_df = pd.DataFrame([
        [
            metrics.homogeneity_score(true_label_nc, pred_walktrap_label_nc),
            metrics.completeness_score(true_label_nc, pred_walktrap_label_nc),
            metrics.v_measure_score(true_label_nc, pred_walktrap_label_nc),
            metrics.adjusted_rand_score(true_label_nc, pred_walktrap_label_nc),
            metrics.adjusted_mutual_info_score(true_label_nc, pred_walktrap_label_nc)
        ],
        [
            metrics.homogeneity_score(true_label_nc, pred_fastgreedy_label_nc),
            metrics.completeness_score(true_label_nc, pred_fastgreedy_label_nc),
            metrics.v_measure_score(true_label_nc, pred_fastgreedy_label_nc),
            metrics.adjusted_rand_score(true_label_nc, pred_fastgreedy_label_nc),
            metrics.adjusted_mutual_info_score(true_label_nc, pred_fastgreedy_label_nc)
        ],
        [
            metrics.homogeneity_score(true_label_lc, pred_walktrap_label_lc),
            metrics.completeness_score(true_label_lc, pred_walktrap_label_lc),
            metrics.v_measure_score(true_label_lc, pred_walktrap_label_lc),
            metrics.adjusted_rand_score(true_label_lc, pred_walktrap_label_lc),
            metrics.adjusted_mutual_info_score(true_label_lc, pred_walktrap_label_lc)
        ],
        [
            metrics.homogeneity_score(true_label_lc, pred_fastgreedy_label_lc),
            metrics.completeness_score(true_label_lc, pred_fastgreedy_label_lc),
            metrics.v_measure_score(true_label_lc, pred_fastgreedy_label_lc),
            metrics.adjusted_rand_score(true_label_lc, pred_fastgreedy_label_lc),
            metrics.adjusted_mutual_info_score(true_label_lc, pred_fastgreedy_label_lc)
        ],
    ],
    index=["No-costraint WalkTrap", "No-costraint FastGreedy", "List-costraint WalkTrap", "List-costraint FastGreedy"],
    columns=["Homogeneity", "Completeness", "V-Measure core", "Adjusted Rand index", "Mutual Information"])

metrics_df

,Homogeneity,Completeness,V-Measure core,Adjusted Rand index,Mutual Information
No-costraint WalkTrap,0.226406,0.833094,0.356050,0.113301,0.214917
No-costraint FastGreedy,0.441698,0.500512,0.469269,0.261725,0.430215
List-costraint WalkTrap,0.188833,0.506902,0.275162,0.003457,0.175389
List-costraint FastGreedy,0.444525,0.444464,0.444494,0.175854,0.432273
